Imports

In [84]:
from pyspark.sql import SparkSession
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql.window import Window #for ranking
from pyspark.sql.functions import lit, mean, stddev_pop
from pyspark.sql.functions import collect_set, collect_list
from pyspark.sql.functions import struct
from pyspark.sql.functions import slice
from pyspark.sql.functions import col
from pyspark.sql.functions import desc
from pyspark.sql.functions import udf
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql.types import DecimalType, ArrayType, IntegerType, FloatType
import pyspark.sql.functions as F
import findspark
from pyspark.sql.functions import avg, broadcast, when

Define cosine similarity and weighted avg functions

In [85]:
# cosine similarity function
def cosine_similarity_udf(a, b):
    dot_product = sum([x * y for x, y in zip(a, b)])
    norm_a = sum([x**2 for x in a])**0.5
    norm_b = sum([x**2 for x in b])**0.5
    return dot_product / (norm_a * norm_b)


def weighted_avg_features(ratings, combined_vectors):
    if not ratings or not combined_vectors:
        return []

    weighted_sum = [0] * len(combined_vectors[0])
    total_weight = 0

    for rating, combined_vector in zip(ratings, combined_vectors):
        weight = float(rating)
        total_weight += weight
        weighted_sum = [ws + weight * f for ws, f in zip(weighted_sum, combined_vector)]

    if total_weight == 0:
        return weighted_sum

    return [ws / total_weight for ws in weighted_sum]

Load Dataset

In [86]:
findspark.init()
spark = SparkSession.builder.appName('ReadMySQL') \
.config("spark.driver.memory", "32g") \
.config("spark.sql.pivotMaxValues", "1000000") \
.config("spark.jars", "C:\\Program Files (x86)\\MySQL\\Connector J 8.0\\mysql-connector-j-8.0.32.jar") \
.getOrCreate()

# sql = "select * from 01_sampled_games_2v2 WHERE playtime_forever IS NOT NULL AND playtime_forever > 0"
sql = """
SELECT p.steamid, p.appid, p.playtime_2weeks, p.playtime_forever, p.dateretrieved, g.genre, d.Developer, pb.Publisher
FROM 01_sampled_games_2v2 AS p
JOIN games_genres AS g ON p.appid = g.appid
JOIN games_developers AS d ON p.appid = d.appid
JOIN games_publishers AS pb ON p.appid = pb.appid
WHERE p.playtime_forever IS NOT NULL AND p.playtime_forever > 0
"""
database = "steam"
user = "root"
password = "root"
server = "127.0.0.1"
port = 3333
jdbc_url = f"jdbc:mysql://{server}:{port}/{database}"
jdbc_driver = "com.mysql.cj.jdbc.Driver"

# Create a data frame by reading data from Oracle via JDBC
df = spark.read.format("jdbc") \
    .option("url", jdbc_url) \
    .option("query", sql) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", jdbc_driver) \
    .load()

df = df.drop("playtime_2weeks", "dateretrieved")

In [87]:
# count number of rows in the dataframe
row_count = df.count()
# print the row count
print("Dataframe has", row_count, " rows.")
df.show()

Dataframe has 212589  rows.
+-----------------+-----+----------------+----------+--------------------+--------------------+
|          steamid|appid|playtime_forever|     genre|           Developer|           Publisher|
+-----------------+-----+----------------+----------+--------------------+--------------------+
|76561197960268000|  300|             109|    Action|               Valve|               Valve|
|76561197960268000| 1300|              94|    Action|Ritual Entertainment|Ritual Entertainment|
|76561197960268000| 2100|             110|    Action|      Arkane Studios|             Ubisoft|
|76561197960268000| 2100|             110|       RPG|      Arkane Studios|             Ubisoft|
|76561197960268000| 4000|             152|     Indie|   Facepunch Studios|               Valve|
|76561197960268000| 4000|             152|Simulation|   Facepunch Studios|               Valve|
|76561197960268000| 2600|              59|    Action|        Troika Games|          Activision|
|76561197960

Build item profiles

In [88]:
# build the item profiles
# Group the data by 'appid' and collect the genres for each game into a list
games_genres_df = df.groupBy("appid").agg(collect_set("genre").alias("genres"))
# Group the data by 'appid' and collect the developers for each game into a list
games_developers_df = df.groupBy("appid").agg(collect_set("Developer").alias("developers"))
# Group the data by 'appid' and collect the publishers for each game into a list
games_publishers_df = df.groupBy("appid").agg(collect_set("Publisher").alias("publishers"))

# Create a list of unique genres
unique_genres = sorted(df.select("genre").distinct().rdd.flatMap(lambda x: x).collect())
# Create a list of unique developers
unique_developers = sorted(df.select("Developer").distinct().rdd.flatMap(lambda x: x).collect())
# Create a list of unique publishers
unique_publishers = sorted(df.select("Publisher").distinct().rdd.flatMap(lambda x: x).collect())


# Define a UDF to create a binary vector for each game's genres
@udf(returnType=ArrayType(IntegerType()))
def genre_vector(genres):
    return [1 if genre in genres else 0 for genre in unique_genres]

# Define a UDF to create a binary vector for each game's developer
@udf(returnType=ArrayType(IntegerType()))
def developer_vector(developers):
    return [1 if developer in developers else 0 for developer in unique_developers]

# Define a UDF to create a binary vector for each game's publisher
@udf(returnType=ArrayType(IntegerType()))
def publisher_vector(publishers):
    return [1 if publisher in publishers else 0 for publisher in unique_publishers]


# Add a new column 'genre_vector' to the DataFrame
# the genre vector will now have a 1 for each genre that the game belongs to
games_genres_df = games_genres_df.withColumn("genre_vector", genre_vector("genres"))
# Add a new column 'developer_vector' to the DataFrame
games_developers_df = games_developers_df.withColumn("developer_vector", developer_vector("developers"))
# Add a new column 'publisher_vector' to the DataFrame
games_publishers_df = games_publishers_df.withColumn("publisher_vector", publisher_vector("publishers"))

# games_genres_df.show(truncate=False)
# Join the main DataFrame with the games_genres_df on appid to include the genre_vector
df = df.join(broadcast(games_genres_df.select("appid", "genre_vector")), on="appid")
# Join the main DataFrame with the games_developers_df
df = df.join(broadcast(games_developers_df.select("appid", "developer_vector")), on="appid")
# Join the main DataFrame with the games_publishers_df
df = df.join(broadcast(games_publishers_df.select("appid", "publisher_vector")), on="appid")

df.show()

+-----+-----------------+----------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+
|appid|          steamid|playtime_forever|     genre|           Developer|           Publisher|        genre_vector|    developer_vector|    publisher_vector|
+-----+-----------------+----------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  300|76561197960268000|             109|    Action|               Valve|               Valve|[1, 0, 0, 0, 0, 0...|[0, 0, 0, 0, 0, 0...|[0, 0, 0, 0, 0, 0...|
| 1300|76561197960268000|              94|    Action|Ritual Entertainment|Ritual Entertainment|[1, 0, 0, 0, 0, 0...|[0, 0, 0, 0, 0, 0...|[0, 0, 0, 0, 0, 0...|
| 2100|76561197960268000|             110|    Action|      Arkane Studios|             Ubisoft|[1, 0, 0, 0, 0, 0...|[0, 0, 0, 0, 0, 0...|[0, 0, 0, 0, 0, 0...|
| 2100|76561197960268000|             110|    

Build the user profile

In [89]:
# Calculate the per-game mean and standard deviation of the playtime column
game_stats = df.filter(col("playtime_forever") > 0).groupBy("appid").agg(
    mean("playtime_forever").alias("game_mean_playtime"),
    stddev_pop("playtime_forever").alias("game_stddev_playtime")
)

# Calculate the overall playtime average
overall_playtime_avg = df.filter(col("playtime_forever") > 0).agg(avg("playtime_forever")).collect()[0][0]

# Calculate the per-steamid playtime average
user_playtime_avg = df.filter(col("playtime_forever") > 0).groupBy("steamid").agg(avg("playtime_forever")).withColumnRenamed("avg(playtime_forever)", "user_playtime_avg")

# Join the user_playtime_avg dataframe with the main dataframe
df = df.join(user_playtime_avg, "steamid")

# Join the game_stats dataframe with the main dataframe
df = df.join(game_stats, "appid")

# Calculate the scaling factor based on the ratio of user playtime to overall playtime
df = df.withColumn("scaling_factor", col("user_playtime_avg") / overall_playtime_avg)

# Calculate the adjusted cut points
df = df.withColumn("cut_point_1", when(col("game_mean_playtime") - (col("game_stddev_playtime") * 0.5 * col("scaling_factor")) > 0, col("game_mean_playtime") - (col("game_stddev_playtime") * 0.5 * col("scaling_factor"))).otherwise(0))
df = df.withColumn("cut_point_2", col("game_mean_playtime") * col("scaling_factor"))
df = df.withColumn("cut_point_3", col("game_mean_playtime") + (col("game_stddev_playtime") * 0.5 * col("scaling_factor")))
df = df.withColumn("cut_point_4", col("game_mean_playtime") + col("game_stddev_playtime") * col("scaling_factor"))

# Assign ratings based on adjusted cut points
df = df.withColumn(
    "ratings",
    when(col("playtime_forever") <= col("cut_point_1"), lit(1))
    .when((col("playtime_forever") > col("cut_point_1")) & (col("playtime_forever") <= col("cut_point_2")), lit(2))
    .when((col("playtime_forever") > col("cut_point_2")) & (col("playtime_forever") <= col("cut_point_3")), lit(3))
    .when((col("playtime_forever") > col("cut_point_3")) & (col("playtime_forever") <= col("cut_point_4")), lit(4))
    .otherwise(lit(5))
)

# Update the user profile calculation to use the new ratings column
user_aggregated_data = df.groupBy("steamid").agg(
    collect_list("genre_vector").alias("genres_list"),
    collect_list("ratings").alias("ratings_list")
)
# Show df with new changes(without genres)
# Drop the genre_vector and genre columns from the DataFrame
df_without_info = df.drop("genre_vector", "genre", "developer_vector", "Developer", "publisher_vector", "Publisher")

# Show the DataFrame without the genre_vector and genre columns
df_without_info.show()
#df.show()

AnalysisException: Column 'ratings' does not exist. Did you mean one of the following? [ratings_adj, appid, genre, steamid, Publisher, cut_point_1, cut_point_2, cut_point_3, cut_point_4, Developer, scaling_factor, genre_vector, playtime_forever, user_playtime_avg, developer_vector, publisher_vector, game_mean_playtime, game_stddev_playtime];
'Aggregate [steamid#12108], [steamid#12108, collect_list(genre_vector#12200, 0, 0) AS genres_list#12669, collect_list('ratings, 0, 0) AS ratings_list#12671]
+- Project [appid#12109L, steamid#12108, playtime_forever#12111L, genre#12113, Developer#12114, Publisher#12115, genre_vector#12200, developer_vector#12205, publisher_vector#12210, user_playtime_avg#12446, game_mean_playtime#12363, game_stddev_playtime#12373, scaling_factor#12551, cut_point_1#12565, cut_point_2#12580, cut_point_3#12596, cut_point_4#12613, CASE WHEN (cast(playtime_forever#12111L as double) <= cut_point_1#12565) THEN 1 WHEN ((cast(playtime_forever#12111L as double) > cut_point_1#

Build user profile off this

In [ ]:
# combine vectors

# Define a UDF to combine the vectors
@udf(returnType=ArrayType(IntegerType()))
def combined_vector(genre_vector, developer_vector, publisher_vector):
    return genre_vector + developer_vector + publisher_vector

# Add 'combined_vector' to the DataFrame
df = df.withColumn("combined_vector", combined_vector("genre_vector", "developer_vector", "publisher_vector"))

# Update the user profile calculation to use the new ratings column and include developer and publisher vectors
user_aggregated_data = df.groupBy("steamid").agg(
    collect_list("combined_vector").alias("combined_vectors_list"),  
    collect_list("ratings").alias("ratings_list")
)

# Define a UDF to calculate the weighted average of genre vectors
weighted_avg_features_udf = udf(weighted_avg_features, ArrayType(FloatType()))

# Calculate the user profile as the weighted average of rated item profiles (combined genre, developer, and publisher vectors)
user_profiles = user_aggregated_data.withColumn("user_profile", weighted_avg_features_udf("ratings_list", "combined_vectors_list"))

user_profiles.show()

+-----------------+---------------------+--------------------+--------------------+
|          steamid|combined_vectors_list|        ratings_list|        user_profile|
+-----------------+---------------------+--------------------+--------------------+
|76561197960271000| [[1, 0, 0, 0, 0, ...|                 [2]|[1.0, 0.0, 0.0, 0...|
|76561197960334000| [[1, 0, 0, 0, 0, ...|[4, 2, 2, 2, 2, 1...|[1.0, 0.2, 0.0, 0...|
|76561197960338000| [[1, 0, 0, 0, 0, ...|                 [2]|[1.0, 0.0, 0.0, 0...|
|76561197960342000| [[1, 0, 0, 0, 0, ...|[2, 2, 2, 2, 2, 2...|[1.0, 0.5555556, ...|
|76561197960425000| [[0, 0, 0, 0, 0, ...|           [2, 2, 2]|[0.33333334, 0.0,...|
|76561197960458000| [[1, 0, 0, 0, 0, ...|[2, 2, 2, 1, 1, 1...|[1.0, 0.0, 0.0, 0...|
|76561197960476000| [[1, 0, 0, 0, 0, ...|              [2, 2]|[1.0, 0.0, 0.0, 0...|
|76561197960884000| [[1, 0, 0, 0, 0, ...|[1, 1, 1, 1, 5, 5...|[1.0, 0.29787233,...|
|76561197961103000| [[1, 0, 0, 0, 0, ...|     [2, 2, 2, 2, 2]|[0.8, 0.0, 0.0

Cosine similarity

In [ ]:
# prediction heuristics
# calculate cosine distance of an item and user profile

# 1. create udf for cosine similarity
cosine_similarity = udf(cosine_similarity_udf, FloatType())
# create dataframe with combined vectors
games_combined_vectors_df = df.select("appid", "combined_vector").distinct()
# cross join the the combined vectors with the user_profiles
cross_joined = games_combined_vectors_df.crossJoin(user_profiles)

#limit the number of rows for testing
games_combined_vectors_df = games_combined_vectors_df.limit(10)
user_profiles = user_profiles.limit(10)
#cross join the combined vectors with the user_profiles
cross_joined = games_combined_vectors_df.crossJoin(user_profiles)

# calculate the cosine similarity between each item and user
recommendations = cross_joined.withColumn(
    "similarity", cosine_similarity("combined_vector", "user_profile")
)

# sort based on similarity score
sorted_recommendations = recommendations.sort(desc("similarity"))

sorted_recommendations.show(10)

+------+--------------------+-----------------+---------------------+--------------------+--------------------+----------+
| appid|     combined_vector|          steamid|combined_vectors_list|        ratings_list|        user_profile|similarity|
+------+--------------------+-----------------+---------------------+--------------------+--------------------+----------+
|104900|[1, 1, 0, 0, 0, 0...|76561197960334000| [[1, 0, 0, 0, 0, ...|[4, 2, 2, 2, 2, 1...|[1.0, 0.2, 0.0, 0...|0.55056196|
|246800|[1, 0, 0, 0, 0, 0...|76561197960334000| [[1, 0, 0, 0, 0, ...|[4, 2, 2, 2, 2, 1...|[1.0, 0.2, 0.0, 0...| 0.5471529|
|104900|[1, 1, 0, 0, 0, 0...|76561197960884000| [[1, 0, 0, 0, 0, ...|[1, 1, 1, 1, 5, 5...|[1.0, 0.29787233,...|0.54323024|
| 65700|[1, 0, 0, 0, 0, 0...|76561197960458000| [[1, 0, 0, 0, 0, ...|[2, 2, 2, 1, 1, 1...|[1.0, 0.0, 0.0, 0...| 0.5240003|
|246800|[1, 0, 0, 0, 0, 0...|76561197960884000| [[1, 0, 0, 0, 0, ...|[1, 1, 1, 1, 5, 5...|[1.0, 0.29787233,...| 0.5205856|
|246800|[1, 0, 0

recommender

In [ ]:
# Create a window by steamid and similarity to get ranking
window_spec = Window.partitionBy("steamid").orderBy(desc("similarity"))

ranked_recommendations = sorted_recommendations.withColumn("rank", F.row_number().over(window_spec))

top_10_recommendations = ranked_recommendations.filter(ranked_recommendations.rank <= 10)
top_10_recommendations.show(10)

+------+--------------------+-----------------+---------------------+------------+--------------------+----------+----+
| appid|     combined_vector|          steamid|combined_vectors_list|ratings_list|        user_profile|similarity|rank|
+------+--------------------+-----------------+---------------------+------------+--------------------+----------+----+
|246800|[1, 0, 0, 0, 0, 0...|76561197960271000| [[1, 0, 0, 0, 0, ...|         [2]|[1.0, 0.0, 0.0, 0...|0.28867513|   1|
|242700|[1, 0, 0, 0, 0, 0...|76561197960271000| [[1, 0, 0, 0, 0, ...|         [2]|[1.0, 0.0, 0.0, 0...|0.28867513|   2|
| 65700|[1, 0, 0, 0, 0, 0...|76561197960271000| [[1, 0, 0, 0, 0, ...|         [2]|[1.0, 0.0, 0.0, 0...| 0.2581989|   3|
|104900|[1, 1, 0, 0, 0, 0...|76561197960271000| [[1, 0, 0, 0, 0, ...|         [2]|[1.0, 0.0, 0.0, 0...| 0.2581989|   4|
|  4700|[0, 0, 0, 0, 0, 0...|76561197960271000| [[1, 0, 0, 0, 0, ...|         [2]|[1.0, 0.0, 0.0, 0...|       0.0|   5|
| 10500|[0, 0, 0, 0, 0, 0...|76561197960